In [1]:
import torch
import warnings
warnings.filterwarnings("ignore")

In [8]:
!pip show transformers

Name: transformers
Version: 4.31.0.dev0
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /home/kamal/jupyter_env/lib/python3.11/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft


In [2]:
model_id = "EleutherAI/gpt-neo-1.3B"

from transformers import AutoModelForCausalLM, AutoTokenizer

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [23]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Mon Jul 17 11:30:43 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA Graphics...  Off  | 00000000:01:00.0  On |                  N/A |
|  0

In [5]:
model = AutoModelForCausalLM.from_pretrained(model_id,resume_download=True,
                                            torch_dtype=torch.bfloat16).to("cuda")

In [6]:
text = "Lets do this guys"

In [11]:
encode_text = tokenizer(text=text,return_tensors='pt').to('cuda')

In [12]:
encode_text

{'input_ids': tensor([[  43, 1039,  466,  428, 3730]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1]], device='cuda:0')}

In [13]:
generate_text = model.generate(encode_text['input_ids'],max_new_tokens = 500,)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [14]:
generate_text

tensor([[   43,  1039,   466,   428,  3730,    13,   198,   198,    40,  1101,
           407,  1654,   611,   428,   318,   262,   826,  1295,   284,  1281,
           428,    11,   475,   314,  1101,   257,   649,  2888,   994,   290,
           314,  1101,  2111,   284,   651,   617,  1037,    13,   314,  1101,
          2111,   284,   651,   616,  2832,   319,   257,  4866,   286,   262,
           983,   290,   314,  1101,  1719,   257,  1327,   640,  4917,   340,
            13,   314,  1053,   587, 10342,   329,   340,   329,   257,   981,
           783,   290,   314,   460,   470,  1283,   284,  1064,   340,    13,
           314,  1053,   587,  2045,   329,   340,   319,   262,  5230,   290,
           314,  1053,   587,  2045,   319,   262,   983,   338,  3052,    13,
           314,  1053,   587,  2045,   329,   340,   319,   262,   983,   338,
          3052,   290,   314,  1053,   587,  2045,   319,   262,   983,   338,
          3052,    13,   314,  1053,   587,  2045,  

In [15]:
decode_text = tokenizer.batch_decode(generate_text)

In [16]:
decode_text

["Lets do this guys.\n\nI'm not sure if this is the right place to post this, but I'm a new member here and I'm trying to get some help. I'm trying to get my hands on a copy of the game and I'm having a hard time finding it. I've been searching for it for a while now and I can't seem to find it. I've been looking for it on the internet and I've been looking on the game's website. I've been looking for it on the game's website and I've been looking on the game's website. I've been looking for it on the game's website and I've been looking on the game's website. I've been looking for it on the game's website and I've been looking on the game's website. I've been looking for it on the game's website and I've been looking on the game's website. I've been looking for it on the game's website and I've been looking on the game's website. I've been looking for it on the game's website and I've been looking on the game's website. I've been looking for it on the game's website and I've been look

In [18]:
del model
torch.cuda.empty_cache()

In [20]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

In [21]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

In [9]:
tokenizer.mask_token

Using mask_token, but it is not set yet.


In [10]:
sequence = "This is a nice way of learning Transformers" 

In [16]:
encoded_input = tokenizer(
    sequence,
    padding='max_length',
    max_length=512,
    truncation=True,
    return_tensors="pt",
)
decoder_input = tokenizer(
    sequence,
    padding='max_length',
    max_length=512,
    truncation=True,
    return_tensors="pt",
)

In [24]:
encoder_output = model.generate(input_ids=encoded_input["input_ids"], 
                                decoder_input_ids=decoder_input["input_ids"],max_length=613)

In [25]:
output = tokenizer.decode(
    encoder_output[0],
    skip_special_tokens=True
)

In [26]:
output

'This is a nice way of learning Transformers This is a nice way to learn Transformers: The Transformers: The Movie is a great way to learn the history of the Transformers.'